# Display map with GeoJSON layers

https://ipyleaflet.readthedocs.io/en/latest/api_reference/geo_json.html


## Configuration

In [1]:
# Specify the locality whose OSM file we want to load

locality = 'Mount Eliza Sample'
margin   = 20
zoom     = 14

## Load required modules

In [2]:
import json
import os
import sys

from ipyleaflet import Map, Marker, GeoJSON

import pandas as pd

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.detection_map as detection_map

## Display map of detected points with boundary

In [3]:
# Version of locality strings with underscores
locality_clean  = locality.replace(' ', '_')
locality_margin = '{0:s}_{1:d}m'.format(locality_clean, margin)

In [4]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(
    os.path.join(
        os.pardir,
        'data_sources',
        'Locality_{0:s}_margin.geojson'.format(locality_clean)
    )
)

center = detection_map.get_centroid(margin_data)

m_detect = Map(center=center, zoom=zoom)
m_detect.add_layer(margin_layer)

# Load detected points and add to map

count = detection_map.load_points(m_detect, os.path.join(os.pardir, 'detections', locality_margin, 'detection_log.csv'))
print('Point count: ' + str(count))

# Display the map
m_detect

Point count: 126


Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

## Display map of detected lines with boundary

In [5]:
# Version of locality strings with underscores
locality_clean  = locality.replace(' ', '_')
locality_margin = '{0:s}_{1:d}m'.format(locality_clean, margin)

detected_filename = os.path.join(
    module_path_root,
    'detections',
    locality_margin,
    'detected_points.geojson'
)

In [6]:
# Load boundary of the sample area, so we can draw it on the map
margin_layer, margin_data = detection_map.load_layer(
    os.path.join(
        os.pardir,
        'data_sources',
        'Locality_{0:s}_margin.geojson'.format(locality_clean)
    )
)

center = detection_map.get_centroid(margin_data)

# Load lines that we drew from the detected points
detected_layer, detected_data = detection_map.load_layer(detected_filename, color='red')

# Build the map
m_detect2 = Map(center=center, zoom=zoom)
m_detect2.add_layer(margin_layer)
m_detect2.add_layer(detected_layer)

# Display the map
m_detect2

Map(center=[-38.169, 145.102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…